In [ ]:
# import ipywidgets as widgets
# from hublib.ui import FileUpload, PathSelector, Download
# import hublib.ui as ui
# import os
# import re
# import sys
# from IPython.display import display, clear_output
# from ipywidgets import Button, Layout
# from src.kernel_methods import *
# from src.neural_network import *
# from src.gui import *
# a = tabs()

# from src.data_tool import *
# import matplotlib.pyplot as plt
# from PIL import Image

# import matplotlib.image as mpimg


In [1]:
import ipywidgets as widgets
from hublib.ui import FileUpload, PathSelector, Download
import hublib.ui as ui
import os
import re
import sys
from IPython.display import display, clear_output
from ipywidgets import Button, Layout
from src.kernel_methods import *
from src.neural_network import *
from src.data_tool import *
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib notebook

import matplotlib.image as mpimg

Materials, Features, Labels = read_data_global()

class tabs():
    
    def __init__(self, tablist= ['Material Selction', 'Method Selection', 'Design', 'MA-VELP'], Materails=['BPY', 'TF']):
        """
        Initialize a tab with multiple childrens (len(tablist))
        It will generate each tab with its own function. Ensure that functions are defined.
        
        """        
        self.tablist = tablist
        self.tab = widgets.Tab()
        
        
        self.tab.children = ()
        
        self.Materials_List = ['BPY', 'PY','PF6','TF','TFSI','DCA']
        self.Materials = Materails
        
        for name in self.tablist:
            if name == 'MA-VELP':
                self.tab.children += (self.generate_docs(),)
            elif name == 'Material Selction':
                self.tab.children += (self.generate_materials(),)
            elif name == 'Method Selection':
                self.tab.children += (self.generate_method(), )
            elif name == 'Design':
                self.tab.children += (self.generate_design(),)
            else:
                raise NameError('Wrong Tab name inserted: ('+ name+')')
                
        for cnt, name in enumerate(self.tablist):
            self.tab.set_title(cnt, name)
        
        display(self.tab)
    
    def generate_docs(self):
         
        discription_text = "Machine-Learned Assisted Virtual Exfoliation in Liquid Phase(MA-VELP):\n\n\n \n\
        Based on the dataset obtained from high-throughput computational study, MAVELP employes machine learning algorithms to screen for optimal solvent based on the user's material selection for exfoliation process via liquid phase. Currently, MAVELP uses potential of mean force as the selection criterion! MAVELP developed at University of Illinois at Urbana-Champaign under Nano-Manufacturing Group is pushing the boundaries of Exfoliation Process Solvent Design! \
            \n \n \n \n \n \nFor assitant contact: moradza2@illinois.edu"
            
        output =  widgets.Output()
        with output:
            print(discription_text)
            print("\nDevelopers: Alireza Moradzadeh, Darren Adams, and Narayana R. Aluru")
                
            
            
        return widgets.VBox([output], layout={'height': '400px'})
    
    
    def generate_method(self):
        style = {'description_width': 'initial'}
        style2 = {'description_width': '150px'}
        self.method = widgets.Dropdown(
            options=[('Neural Network', 1), ('Kernel Ridge Regression', 2)],
            value=2,
            description='Method:', style=style, layout={'width': 'max-content'})
        self.submit_buttom = widgets.Button(description="Run regression")
        self.progress = widgets.FloatProgress(
                    value=0.0,
                    min=0,
                    max=10.0,
                    step=0.1,
                    description='Progress:',
                    bar_style='info',
                    orientation='horizontal',visibility = 'hidden', style=style, layout={'width': '250px'})
        
        self.progress.layout.visibility = 'hidden'
        
        output = widgets.Output()#layout={'width': '300px', 'height': '300px'})
        def on_button_clicked(b):
            
            output.clear_output()
            if self.method.value == 2:
                self.progress.layout.visibility = 'hidden'
                with output:
                    print("Starting to fit!")
                
                ml = Kernel_Optimization(kernel_type=self.method_options_kr_type.value , CV=self.method_options_kr_cv.value, X=Features, y=Labels)
                
                with output:
                    print("Kernel is fitted!")
                    loss = ml.score()
                    print("loss: ", np.round(loss))
            elif self.method.value == 1:
                
                output.clear_output()
                self.progress.value = 0    
                self.progress.layout.visibility = 'visible'
                
                ml = DNN(n_dims=6, dnn_features=[self.methods_options_dnn_h1.value, self.methods_options_dnn_h2.value,\
                                                  self.methods_options_dnn_kp.value, self.methods_options_dnn_lr.value,\
                                                  self.method_options_dnn_nsteps.value], X=Features,y=Labels )
                
                for epoch in range(self.method_options_dnn_nsteps.value):
                    
                    feed = {ml.x_placeholder: ml.X, ml.y_placeholder: ml.y,\
                            ml.kp_placeholder:self.methods_options_dnn_kp.value, ml.lr_placeholder: self.methods_options_dnn_lr.value}
                    if epoch == 0:
                        ml.session.run(ml.train_opt, feed_dict=feed)
                    k = (10*(epoch+1)/self.method_options_dnn_nsteps.value)
                    self.progress.value = k
                    loss = np.random.uniform(0,1,1)
                    if epoch % 2000 == 0 :
                        output.clear_output()
                with output:
                    fig = plt.figure(1, figsize=(5,4))
                    
                    
                    plt.plot(np.linspace(0,1,10), np.random.uniform(-1,1,10),'r-o')
#                     fig.canvas.layout.width = '200px'
#                     fig.canvas.layout.height= '200px'
                    plt.show()
                    
                    
            elif self.method.value == 3:
                
                with output:
                    print("This method is not implemented yet!")
                
                

        self.submit_buttom.on_click(on_button_clicked)
        
        Method_Box = widgets.VBox([self.method, self.submit_buttom , self.progress, output], layout={'width': '550px'})
        # Kernel Rigde Regression Options
        self.method_options_kr_cv = widgets.IntSlider(value=2, min=2,max=10, step=1,
            description='Cross-Validation:',
            disabled=False, continuous_update=False,
            orientation='horizontal',readout=True,
            readout_format='d', style=style)
        
        self.method_options_kr_type = widgets.Dropdown(
            options=['RBF', 'ExpSineSquared'],
            value='RBF',
            description='Kernel Type:', style={'description_width': '100px'}, layout={'width': '260px'})
        
        # Neural Network Option
        
        self.methods_options_dnn_h1 = widgets.IntSlider(value=15, min=1,max=100, step=1,
            description='Hidden Layer 1 Nodes:',
            disabled=False, continuous_update=False,
            orientation='horizontal',readout=True,
            readout_format='d', style=style2)#,  layout={'width': '260px'})
        
        self.methods_options_dnn_h2 = widgets.IntSlider(value=5, min=1,max=50, step=1,
            description='Hidden Layer 2 Nodes:',
            disabled=False, continuous_update=False,
            orientation='horizontal',readout=True,
            readout_format='d', style=style2)#, layout={'width': '260px'})
        
        self.methods_options_dnn_h1_f = widgets.Dropdown(
            options=[('tanh', 1), ('sigmoid', 2), ('tanh+sigmoid', 3)],
            value=1,
            description='Activation Layer 1:', style=style2, layout={'width': '260px'})
        
        self.methods_options_dnn_h2_f = widgets.Dropdown(
            options=[('tanh', 1), ('sigmoid', 2), ('tanh+sigmoid', 3)],
            value=1,
            description='Activation Layer 2:', style=style2, layout={'width': '260px'})
        self.methods_options_dnn_loss = widgets.Dropdown(
            options=[('MSE', 1), ('MAE', 2)],
            value=1,
            description='Loss function: ', style=style2, layout={'width': '260px'})
        self.methods_options_dnn_optimizer = widgets.Dropdown(
            options=[('Adagrad', 1), ('SGD', 2), ('Adam', 3)],
            value=1,
            description='Optimizer: ', style=style2, layout={'width': '260px'})
        
        self.methods_options_dnn_lr = widgets.BoundedFloatText(
                value=.0001,
                min=0,
                max=1.0,
                step=0.000001,
                description='Learning Rate:',
                disabled=False, style=style2, layout={'width': '260px'})
        
        self.methods_options_dnn_kp = widgets.BoundedFloatText(
                value=0.001,
                min=0,
                max=1.0,
                step=0.001,
                description='Keep Probability:',
                disabled=False, style=style2, layout={'width': '260px'})
        self.method_options_dnn_nsteps = widgets.IntText(value=200,step=1,
            description='Training steps:',
            disabled=False, continuous_update=False,
            orientation='horizontal',readout=True,
            readout_format='d', style=style2)
        
        
        # Random Forest Option
        self.method_options_rf_max_depth = widgets.IntSlider(value=3, min=1,max=10, step=1,
            description='Depth of Tree:',
            disabled=False, continuous_update=False,
            orientation='horizontal',readout=True,
            readout_format='d', style=style2)
        
        self.method_option_rf_n_estimators = widgets.BoundedIntText(
                value=100,
                min=1,
                max=500,
                step=1,
                description='Num. Estimators:',
                disabled=False, style=style2, layout={'width': '260px'})
        
        V_box_KR = widgets.VBox([self.method_options_kr_cv,self.method_options_kr_type], layout={'width': '325px'})
        V_box_DL = widgets.VBox([self.methods_options_dnn_h1, self.methods_options_dnn_h2, self.methods_options_dnn_h1_f, self.methods_options_dnn_h2_f, \
                                 self.methods_options_dnn_loss,self.methods_options_dnn_optimizer, self.methods_options_dnn_lr, self.method_options_dnn_nsteps,\
                                 self.methods_options_dnn_kp], layout={'width': '400px'})
        V_box_RF = widgets.VBox([self.method_options_rf_max_depth, self.method_option_rf_n_estimators], layout={'width': '450px'})
        
        V_box_DL.layout.visibility = 'hidden'
        V_box_KR.layout.visibility = 'visible'
        V_box_RF.layout.visibility = 'hidden'
        
        
        def on_selection_button(b):
            if self.method.value == 1:
                V_box_DL.layout.visibility = 'visible'
                V_box_RF.layout.visibility = 'hidden'
                V_box_KR.layout.visibility = 'hidden'
                
            elif self.method.value == 2:
                V_box_DL.layout.visibility = 'hidden'
                V_box_RF.layout.visibility = 'hidden'
                V_box_KR.layout.visibility = 'visible'
                
            elif self.method.value == 3:
                V_box_DL.layout.visibility = 'hidden'
                V_box_RF.layout.visibility = 'visible'
                V_box_KR.layout.visibility = 'hidden'
                
        self.method.observe(on_selection_button)
        
        #return widgets.HBox([Method_Box, V_box_KR ,V_box_DL ,V_box_RF ], layout={'height': '400px', 'width': '1200px'})
        return widgets.HBox([Method_Box, V_box_KR ,V_box_DL], layout={'height': '600px', 'width': '1200px'})

    def generate_materials(self):
        style = {'description_width': 'initial'}
        self.CationCell =widgets.SelectMultiple(
                options=['BPY', 'PY'],
                value=['BPY'],
                #rows=10,
                description='Cations',
                disabled=False
            )
        self.AnionCell =widgets.SelectMultiple(
                options=['PF6', 'TF', 'TFSI', 'DCA'],
                value=['TF'],
                #rows=10,
                description='Anions',
                disabled=False
            )
           
        items_layout = Layout( width='50%')     # override the default width of the button to 'auto' to let the button grow

        box_layout = widgets.Layout(display='flex',
                                flex_flow='row',
                                align_items='stretch',
                                border='None',
                                width='100%')
        self.button = widgets.Button(description="Click After Material Selection!", style=style, layout={'width': '260px'})
        output = widgets.Output()

        def on_button_clicked(b):
            
            output.clear_output()
            self.Materials = []
            with output:
                output_str = "Cations: "
                self.Materials = [v for v in self.CationCell.value]
                self.Materials.extend([v for v in self.AnionCell.value])
                for v in self.CationCell.value:
                    output_str += v+", "
                print(output_str[:-2])
                output_str = "Anions: "
                for v in self.AnionCell.value:
                    output_str += v+", "
                print(output_str[:-2])
            t = list(self.tab.children)
            t[2] = self.generate_design()
            self.tab.children = t
            
        self.button.on_click(on_button_clicked)
            
        self.Materials = [v for v in self.CationCell.value]
        self.Materials.extend([v for v in self.AnionCell.value])

        items = [self.CationCell, self.AnionCell, self.button, output]

        box = widgets.Box(children=items, layout=box_layout)
            
        return widgets.VBox([box], layout={'height': '600px'})
        
    def generate_design(self):
        """
        Design Tab controller
        """
        " List of possible methods: ‘Nelder-Mead’, ‘Powell’, ‘CG’, ‘BFGS’,‘Newton-CG’,‘L-BFGS-B’,‘TNC’,‘COBYLA’,‘SLSQP’,‘trust-constr’,‘dogleg’,‘trust-ncg’,‘trust-exact’,‘trust-krylov’ "
        style = {'description_width': '150px'}
        
        self.Optimization_method = widgets.Dropdown(
            options=[('Nelder-Mead', 1), ('L-BFGS-B', 2)],
            value=1,
            description='Optimization method:', style=style)
        
        self.Optimization_submit = widgets.Button(description="Submit", style =style )
        
        def submit_call_back(b):
            print(f(1))
            
        self.Optimization_submit.on_click(submit_call_back)
        
        Optimizer_Box = widgets.VBox([self.Optimization_method, self.Optimization_submit])#, layout={'height': '300px', 'width': '400px'}) 
        
        Warning_ouput = widgets.Output()
        
        with Warning_ouput:
            print("Cations or anions molar fractions sum should be equal to one otherwise their ratio is going to be used!")
        self.Prediction_Box = ()
        
        
        for cnt, name in enumerate(self.CationCell.value):
            if cnt == 0 :
                self.Prediction_Box += (widgets.BoundedFloatText(value=1.0,min=0,max=1.0,step=0.001,description=name,disabled=False, style=style), )
            else:
                self.Prediction_Box += (widgets.BoundedFloatText(value=0.0,min=0,max=1.0,step=0.001,description=name,disabled=False, style=style), )    

        
        for cnt, name in enumerate(self.AnionCell.value):
            if cnt == 0 :
                self.Prediction_Box += (widgets.BoundedFloatText(value=1.0,min=0,max=1.0,step=0.001,description=name,disabled=False, style=style), )
            else:
                self.Prediction_Box += (widgets.BoundedFloatText(value=0.0,min=0,max=1.0,step=0.001,description=name,disabled=False, style=style), )    
        self.Prediction_Box += (widgets.Button(description="Run", style=style, layout={'width': 'max-content'}), )
        self.Value_box = widgets.Output()
        
        HBoxPrediction = widgets.HBox([widgets.VBox(self.Prediction_Box), widgets.VBox([self.Value_box])])
        def click_button(b):
            self.Value_box.clear_output()
            with self.Value_box:
                sum_molar_cat =  np.sum([self.Prediction_Box[cnt].value for cnt in range(len(self.CationCell.value))])
                for cnt in range(len(self.CationCell.value)):
                    print(np.round(self.Prediction_Box[cnt].value/sum_molar_cat,3), '\n')
                sum_molar_an =  np.sum([self.Prediction_Box[len(self.CationCell.value)+cnt].value for cnt in range(len(self.AnionCell.value))])
                for cnt in range(len(self.AnionCell.value)):
                    print(np.round(self.Prediction_Box[cnt+len(self.CationCell.value)].value/sum_molar_an,3),'\n')
                    
        self.Prediction_Box[-1].on_click(click_button)

        accordion = widgets.Accordion(children=[Optimizer_Box, HBoxPrediction, Warning_ouput],layout={'height': '375px', 'width': '800px'} )
        accordion.set_title(0, 'Optimization of solvent performance')
        accordion.set_title(1, 'Prediction of solvent performance  ')    
        accordion.set_title(2, 'Warning for Prediction of solvent performance  ')    
        
        return widgets.VBox([accordion], layout={'height': '600px'})
        
    def generate_child(self, name):
        """
        generic child addition
        """
        return widgets.VBox([widgets.Text(description=name)], layout={'height': '600px'})

a=  tabs()

<IPython.core.display.Javascript object>

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
# button = widgets.Button(description="Click Me!")
# output2 = widgets.Output()

# display(button, output2)

# def on_button_clicked(b):
#     with output2:
#         print(f(1))

# button.on_click(on_button_clicked)





In [ ]:
# Kr = Kernel_Optimization(CV=2)

In [ ]:
# Kr.kr_func([1,2])
# Kr.X.shape

In [ ]:
# CationCell =widgets.SelectMultiple(
#     options=['BPY', 'PY'],
#     value=['BPY'],
#     #rows=10,
#     description='Cations',
#     disabled=False
# )
# AnionCell =widgets.SelectMultiple(
#     options=['PF6', 'TF', 'TFSI', 'DCA'],
#     value=['TF'],
#     #rows=10,
#     description='Anions',
#     disabled=False
# )
# # display(CationCell,AnionCell)

In [ ]:
# items_layout = Layout( width='40%')     # override the default width of the button to 'auto' to let the button grow

# box_layout = widgets.Layout(display='flex',
#                     flex_flow='row',
#                     align_items='stretch',
#                     border='None',
#                     width='100%')
# button = widgets.Button(description="Show Materials to Me!")
# output = widgets.Output()

# def on_button_clicked(b):
#     output.clear_output()
#     with output:
        
#         output_str = "Cations: "
#         for v in CationCell.value:
#             output_str += v+", "
#         print(output_str[:-2])
#         output_str = "Anions: "
#         for v in AnionCell.value:
#             output_str += v+", "
#         print(output_str[:-2])

# button.on_click(on_button_clicked)


# words = ['correct', 'horse', 'battery', 'staple']

# items = [CationCell, AnionCell, button, output]


# box = widgets.Box(children=items, layout=box_layout)



# display(box)

In [ ]:
# import urllib
# import requests
# f = open('sample.png','wb')
# f.write(requests.get("https://drive.google.com/file/d/1NLQD20pHOeFPyA0FCCGWPaK7XaTQ5qK-/view?usp=sharing").content)
# f.close()

In [ ]:
# f(1)

In [ ]:
# import src.mlp as mlp
# method_to_call = getattr(mlp, 'f')
# result = method_to_call(1)
# result    

In [ ]:
# %matplotlib inline

# # To prevent automatic figure display when execution of the cell ends
# %config InlineBackend.close_figures=False 

# import matplotlib.pyplot as plt
# import numpy as np

# from IPython.html import widgets
# from IPython.display import display,clear_output

# plt.ioff()
# ax=plt.gca()

# out=widgets.Output()



# button = widgets.Button(description='Next')
# vbox = widgets.VBox(children=[])

# def click(event=None):
#     out = widgets.Output()
#     ax.clear()
#     ax.plot(np.random.randn(100),np.random.randn(100),'+')
#     with out:
#         clear_output(wait=True)
#         display(ax.figure)
#     vbox.children = [button, out]

# button.on_click(click)
# click()
# display(vbox)


In [ ]:
# caption = widgets.Label(value='Changes in source values are reflected in target1')
# source, target1 = widgets.FloatSlider(value=1,description='Source'),\
#                   widgets.FloatSlider(description='Target 1')
# source2 = widgets.FloatSlider(description='Source')
# def sum_to_one(cnt):
#     if cnt ==0:
#         return target1.value/(target1.value+ 10 +source.value + source2.value)
#     elif cnt == 1:
#         return (10 +source.value)/(target1.value+source.value + source2.value)
#     elif cnt == 2:
#         return (source2.value)/(target1.value+ 0 +source.value + source2.value)
# # def update_slider(change):
# #     target1.value = source.value

# # target1.observe(update_slider, names='value')

# dl = widgets.dlink((source, 'value'), (target1, 'value'), transform=sum_to_one(cnt=0))
# dl2 = widgets.dlink((source2, 'value'), (target1, 'value'), transform=sum_to_one(cnt=1))
# dl3 = widgets.dlink((source2, 'value'), (source, 'value'), transform=sum_to_one(cnt=2))

# display(caption, source,source2, target1)

In [ ]:
# ?widgets.dlink

In [ ]:
# os.chdir('src')
# np.savetxt('gui.py',np.array([1]))

In [ ]:
# os.getcwd()